In [1]:
import tensorflow as tf


2024-03-05 15:48:55.287327: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-05 15:48:55.287398: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-05 15:48:55.288683: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-05 15:48:55.295819: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-05 15:48:57.167118: W tensorflow/compiler/tf2

In [ ]:
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-1)

iterations = 200
for iter in range(iterations):

    with tf.GradientTape() as tape:
        cost_value = cofiCostFuncV(X, W, b, Ynorm, R, num_users, num_movies, lambda_)

    grads = tape.gradient( cost_value, [X,W,b])

    optimizer.apply_gradients( zip(grads, [X,W,b]))